In [1]:
import pandas as pd
import re
import os
from symspellpy import SymSpell, Verbosity


In [2]:
df = pd.read_csv(r"D:\epita class notes\semester - 3\action learnign\project repository\Hate_speech_detection_using_data_augmentation\Hate_speech_detection_using_data_augmentation\data\original_dataset\labeled_data.csv")
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [3]:
df_filtered = df.head(1000)
df_filtered

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
995,1017,3,0,3,0,1,&#128514;&#128514;&#128514;&#128514; RT @SMASH...
996,1018,3,0,3,0,1,&#128514;&#128514;&#128514;&#128514; bitch if ...
997,1019,3,0,2,1,1,&#128514;&#128514;&#128514;&#128514; these fol...
998,1020,6,0,6,0,1,&#128514;&#128514;&#128514;&#128514;&#128514; ...


In [4]:
def clean_text(text):
    text = text.lower() #converting the tweets to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) #removing url's
    text = re.sub(r'@\w+', '', text)  # removing the username from the tweet
    text = re.sub(r'[^a-zA-Z\s]', '', text) #removing non alpha-numeric character
    text = re.sub(r'\s+', ' ', text) #removing extra space
    text = re.sub(r'\brt\b', '', text) #removing rt from retweeted tweets
    text = re.sub(r'\b[a-zA-Z]*\*+[a-zA-Z]*\b', '[censored]', text) #replacing obscene words with censored
    text = text.strip()
    return text
    

In [10]:
def clean_dataset(df, sym_spell):
    df['cleaned_tweet'] = df['tweet'].apply(clean_text)
    df['corrected_tweet'] = df['cleaned_tweet'].apply(lambda x: correct_spelling(x, sym_spell))
    return df[['count','hate_speech', 'offensive_language', 'neither','corrected_tweet', 'class']]

In [6]:
def initialize_symspell():
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    dictionary_path = r"D:\epita class notes\semester - 3\action learnign\project repository\Hate_speech_detection_using_data_augmentation\Hate_speech_detection_using_data_augmentation\resources\frequency_dictionary_en_82_765.txt"
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    return sym_spell

In [7]:
def correct_spelling(text, sym_spell):
    corrected_words = []
    for word in text.split():
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)  
        else:
            corrected_words.append(word) 
    return ' '.join(corrected_words)

In [8]:
sym_spell = initialize_symspell()

In [11]:
cleaned_df_1 = clean_dataset(df_filtered, sym_spell)
file_name = "labeled_data_cleaned.csv"
output_path = r"D:\epita class notes\semester - 3\action learnign\project repository\Hate_speech_detection_using_data_augmentation\Hate_speech_detection_using_data_augmentation\data\cleaned_dataset"
output_file = os.path.join(output_path, file_name)
cleaned_df_1.to_csv(output_file, index=False)
print(cleaned_df_1) 

C:\Users\edwin victor\AppData\Local\Temp\ipykernel_17144\4234741622.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweet'] = df['tweet'].apply(clean_text)


     count  hate_speech  offensive_language  neither  \
0        3            0                   0        3   
1        3            0                   3        0   
2        3            0                   3        0   
3        3            0                   2        1   
4        6            0                   6        0   
..     ...          ...                 ...      ...   
995      3            0                   3        0   
996      3            0                   3        0   
997      3            0                   2        1   
998      6            0                   6        0   
999      3            1                   2        0   

                                       corrected_tweet  class  
0    as a woman you shouldn't complain about cleani...      2  
1    boy date coldtyga own bad for coffin dat hoe i...      1  
2    dawn you ever fuck a bitch and she start to cr...      1  
3                               she look like a granny      1  
4    th

C:\Users\edwin victor\AppData\Local\Temp\ipykernel_17144\4234741622.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['corrected_tweet'] = df['cleaned_tweet'].apply(lambda x: correct_spelling(x, sym_spell))
